# Matrix Factorization

Okay, so time to use Matrix Factorization. I'll be using Non-Negative Matrix Factorization (NMF). I'm a fan of Non-Negative Matrix Factorization for 2 reasons:
1. There are no negative values, which prevents weird results. Since there is no 'thumbs down' other than not purchasing a product, I don;t want to have negative values spitting out negative results.
2. I prefer the US matrix form, since it allows for manipulation with a K matrix... U is u x k, S is k x s, and K is a k x k matrix you can throw in... if K is I (identity) it has absolutly no effect, but you can maipulate results by tossing numbers on the off-diagonals (for example, in a boom recommender, you can find the group that pregnancy books are in, find the group that stillbirth books are in, and have it negativley influence result for pregnancy books if you're interested in stillbirth books, since that seems like a cruel thing to hit customers with)

There is an issue however... I won't be able to use non-users to validate the way I did above; they need to get a vector at the same time as everyone else... so I need to 'knock out' values in my utility_matrix to fit back in later...but that means I need to grab that again, since I'm in a new notebook...

In [1]:
import pandas as pd
from scipy import sparse

# first load my data...
ratings = pd.read_csv('../data/ratings.csv', delimiter='|', header=None, names=['user_id', 'system_id', 'ratings'])

# get highest user_id & highest system_id
highest_user_id = ratings.user_id.max()
highest_system_id = ratings.system_id.max()

# make a sparse matrix...
utility_matrix = sparse.lil_matrix((highest_user_id + 1, highest_system_id + 1))
# +1 to be able to use actual ids, as opposed to having to make consessions

# of course, now I need to fill it with ratings...
for _, row in ratings.iterrows():
        utility_matrix[row.user_id, row.system_id] = row.ratings

## Validation

Now to remove 20% of my data points and call it the *train_utility_matrix*.  Goal is to make a list of tuples of *user_id* and *system_id*, then set the values at those locations to 0 in a copy of the *utility_matrix*.

In [2]:
from random import shuffle

train_utility_matrix = utility_matrix.copy()
utility_dict = utility_matrix.todok(copy=False)
lst = utility_dict.keys()
shuffle(lst)
cut = int(len(lst)*0.8)
train = lst[:cut]
hold_out = lst[cut:]

# now remove hold_out
for tup in hold_out:
    train_utility_matrix[tup] = 0
    
for tup in hold_out[:5]:
    print utility_matrix[tup], train_utility_matrix[tup]

1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0


## Non-Negative Matrix Factorization

The NMF module from [sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html) does not seem like [a good fit](https://stackoverflow.com/questions/33623144/how-can-i-make-recommendation-model-using-pythons-scikit-learn) as it does not seem to allow much modification of the cost function to account for recommendation systems.  So, I'll need to write my own.
Luckily I did build an NMF class as part of my [Galvanize data science course](https://www.galvanize.com/seattle/data-science), so that's floating around my hard drive/github.
But, I need to change my cost function to the following cost function (thank you [Matt Drury](https://github.com/madrury)):
$$arg: min_{U,V,b^*,b'} \sum_{i, j \in \kappa} (r_{ij} - \mu - b_i^* - b_j' - u_{i:} \cdot v_{:j})^2 + \lambda_1(\parallel u_{i:}\parallel^2 + \parallel v_{:j}\parallel^2) + \lambda_2((b_i^*)^2 + (b_j')^2) $$
Where $r_{ij}$ is user i's rating of item j, $u_{i:}$ is the way user i interacts with our latent features, $v_{j:}$ is how item j is described in terms of our latent features, $b_i^*$ is how much user i deviates from the overall average, and $b_j'$ is how much item j deviates from the overall average, $\mu$.  This gives me a rating function of:
$$r_{ij} \approx \mu + b_i^* + b_j' + u_{i:} \cdot v_{:j}$$ 

Of course, I need  least-squares solution in order to do this.  What I have is [numpy.linalg.lstsq()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.lstsq.html).  This minimizes the following function:
$$arg: min \parallel b - a x \parallel^2  =  min \parallel R - UV \parallel^2 = min_{U,V} \sum_{i, j \in \kappa} (r_{ij} - u_{i:} \cdot v_{:j})^2 $$ 
What we want is the following, if we make a small assumption: Make a matrix B such that $b_{ij} =  b_i^* + b_j'$.  Now, we can make our function the following (assuming $((b_i^*)^2 + (b_j')^2) \approx \parallel b_{ij} \parallel^2$)
$$arg: min \parallel R - UV \parallel^2 + \lambda_1\parallel U \parallel^2 + \lambda_1\parallel V \parallel^2 + \lambda_2\parallel B \parallel^2$$

Since we can [translate](https://stackoverflow.com/questions/27476933/numpy-linear-regression-with-regularization): $\parallel b - a x \parallel^2 + lambda\parallel x \parallel^2$ as: `np.linalg.lstsq(a + lamb, b)`, we can simply solve for the various parts in these terms, which gives us: a = U + $\lambda_1$I and b = R - $\mu$ + ($\lambda_2$ - 1)B.


In [15]:
import numpy as np
from math import sqrt

class NMF_recommender(object):
    '''
    A Non-Negative Matrix Factorization recommender.
    Parameters:
        k - The number of latent features
        max_iter - number of iterations before the Factorization cancels out
        thresh - the threshold, how different the cost function needs to be in
                 in order to kick out of the iterations.
        l1 - lamba_1 in cost function
        l2 - lamba_2 in cost function
        verbose - set True to print out Cost Function during fitting
            
    Attributes:
        k - number of latent features
        iter - max_iter
        thresh - threshold paramater
        l1 - lamba_1
        l2 - lamba_2
    '''

    def __init__(self, k, max_iter=100, thresh=0.0001, l1=0.1, l2=0.1, verbose=False):
        '''
        Initializer.  See class for details.
        '''
        self.k = k
        self.thresh = thresh
        self.iter = max_iter
        self.l1 = l1
        self.l2 = l2
        self.verbose = verbose

    def fit(self, X):
        '''
        Fit function for the recommender system.
        Parameters:
            X - the utility-matrix we are trying to fit to

        Attributes:
            V - utility matrix in array form
            nonzero - a numpy array that is 1 where utility matrix is
                      nonzero and 0 otherwise
            b_star - how much users deviate from average
            b_prime - how much items deviate from average
            b_matrix - a matrix of b_stari - b_primej for all i,j
            cost - the cost function for a recommender
            W - User matrix
            H - Item Matrix
        '''
        # turn utility matrix to an arry
        self.V = X.toarray()
        nonzero = np.where(self.V > 0,1 ,0)
        k  = self.k
        
        # find mu...
        self.mu = np.sum(self.V)/np.count_nonzero(self.V)
        
        # find b_star & b_prime
        self.b_star = np.mean(self.V - self.mu*nonzero, axis=1)
        self.b_prime = np.mean(self.V - self.mu*nonzero, axis=0)
        
        # make b_matrix...
        # first, get a matrix of b_star as columns
        self.b_matrix =  np.column_stack((self.b_star for a in xrange(self.b_prime.shape[0])))
        # add to b_prime as rows...
        self.b_matrix = self.b_matrix + np.row_stack((self.b_prime for a in xrange(self.b_star.shape[0])))
       
        
        self.W = np.random.random_sample((self.V.shape[0], k))
        self.H = np.ones((k, self.V.shape[1]))
        if self.verbose:
            print 'Getting cost...'
        #cost function...
        Q = self.V - self.mu - self.b_matrix
        self.cost = (np.linalg.norm(Q - self.W.dot(self.H)) + 
                         self.l1*(np.linalg.norm(self.W) + np.linalg.norm(self.H)) +
                         self.l2*(self.b_star.dot(self.b_star) + self.b_prime.dot(self.b_prime)))
        if self.verbose:
            print 'Cost starts at: {}'.format(self.cost)
        n = 0
        old_cost = self.cost + 2 * self.thresh
        I = np.identity(k)
        while (n < self.iter) and (abs(old_cost - self.cost) > self.thresh):
            if n % 2 == 0:
                self.H = np.linalg.lstsq(self.W + self.l1, Q + self.l2*self.b_matrix)[0]
                self.H[self.H < 0] = 0
            else:
                self.W = np.linalg.lstsq(self.H.T + self.l1, (Q + self.l2*self.b_matrix).T)[0].T
                self.W[self.W < 0] = 0
            old_cost = self.cost
            self.cost = (np.linalg.norm(Q - self.W.dot(self.H)) + 
                         self.l1*(np.linalg.norm(self.W) + np.linalg.norm(self.H)) +
                         self.l2*(self.b_star.dot(self.b_star) + self.b_prime.dot(self.b_prime)))
            n +=1
            if self.verbose:
                print "Iteration {}: Cost Difference = {}".format(n, abs(old_cost - self.cost))
        return self.W, self.H
    
    def predict(self):
        '''
        Get values for items.                
        Attributes:
            ratings - The returned ratings
        '''
        self.ratings = self.mu + self.b_matrix + self.W.dot(self.H)
        return self.ratings

    def predict_cold(self, v):
        '''
         Fit function for the recommender system.
        Parameters:
            v - a list of tuples describing how user rated items,
                format [(item_1, rating_1), (item_2, rating_2), ..., (item_n, raing_n)]

        Attributes:
            ratings - The returned ratings for all the included users
        '''
        # make sure we have ratings
        try:
            ratings = self.ratings
        except:
            ratings = self.predict()
        pass

    def get_rmse(self, X_full):
        '''
        Assumes a training matrix with missing values was used, checks the values that are in
        X_full but not in self.V and calculates the RMSE.
        Parameters:
            X_full - the utility-matrix that vaules were removed from to get X in our fit

        Attributes:
            Xf - the full utility matric
            rmse - root mean squared error
        '''
         # make sure we have ratings
        ratings = self.predict()
        self.Xf = X_full.toarray()
        X_1 = np.where(self.Xf > 0, 1, 0)
        X_2 = np.where(self.V == 0, 0, 1)
        X_check = X_1 - X_2
        X_check[X_check <= 0] = 0
        self.rmse = sqrt(np.sum((self.Xf[X_check==1] - self.ratings[X_check==1])**2)/float(np.count_nonzero(X_check)))
        return self.rmse

Grid Searching (in an ec2 to allow it to run when I'm not around) gave me the following parameters as 'best':



In [ ]:
nmf = NMF_recommender(4, verbose=True)
nmf.fit(train_utility_matrix)

Getting cost...
Cost starts at: 56731.4481707
Iteration 1: Cost Difference = 32024.5955138
Iteration 2: Cost Difference = 9.86481736393e+15
Iteration 3: Cost Difference = 3.15696361028e+15
Iteration 4: Cost Difference = 4.54519767464e+15
Iteration 5: Cost Difference = 6.51121429751e+14
Iteration 6: Cost Difference = 1.69158572179e+16
Iteration 7: Cost Difference = 518.537702045
Iteration 8: Cost Difference = 66705.6363999
Iteration 9: Cost Difference = 20.34088178
Iteration 10: Cost Difference = 803257.729809
Iteration 11: Cost Difference = 17.2693096879
Iteration 12: Cost Difference = 1245384.08952
Iteration 13: Cost Difference = 0.073382018134
Iteration 14: Cost Difference = 1998013.16445
Iteration 15: Cost Difference = 0.0733792455867
Iteration 16: Cost Difference = 3247420.36189
Iteration 17: Cost Difference = 0.0733777340502
Iteration 18: Cost Difference = 5324281.45968
Iteration 19: Cost Difference = 0.0733761005104
Iteration 20: Cost Difference = 8836582.90385
Iteration 21: Cost